In [1]:
!pip install langgraph
!pip install langchain_community
!pip install langchain
!pip install -U duckduckgo-search
!pip install google-search-results #for serpapi

ERROR: Could not find a version that satisfies the requirement langgrpah (from versions: none)
ERROR: No matching distribution found for langgrpah


In [39]:
from google.colab import userdata
groq_pi_key=userdata.get('groq_api_key')
lang_smith=userdata.get('lang_smith')
serp_api_key = userdata.get("serp_api_key")

In [40]:
import os
os.environ["GROQ_API_KEY"] = groq_pi_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = lang_smith
os.environ["SERPAPI_API_KEY"] = serp_api_key

In [79]:
import langchain
from langchain_community.tools import DuckDuckGoSearchRun
from serpapi import GoogleSearch
from langchain.agents import initialize_agent,Tool
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [11]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=groq_pi_key,
    verbose=True,
)

In [12]:
llm.invoke("Do You know about myself?")

AIMessage(content="I don't have any personal information about you. I'm a large language model, I don't have the ability to know or store information about individual users, including their personal details, preferences, or history. Each time you interact with me, it's a new conversation and I start from a blank slate.\n\nHowever, I can try to get to know you better through our conversation. If you'd like to share something about yourself, I'm all ears (or rather, all text)! What would you like to talk about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 41, 'total_tokens': 148, 'completion_time': 0.389090909, 'prompt_time': 0.004666811, 'queue_time': 0.00252171, 'total_time': 0.39375772}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_564b785073', 'finish_reason': 'stop', 'logprobs': None}, id='run-85669b7a-0ee2-4059-8bdb-9f62db5cc1e2-0', usage_metadata={'input_tokens': 41, 'output_tokens': 107, 'total_tokens'

In [71]:
#initialize the tools
search = DuckDuckGoSearchRun()
tools =[
    Tool(
        name="Duck-Duck-Search-Tool",
        func = search.run,
        description = "useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [72]:
search.invoke("Do you who is current pm of amercia")

"The White House, official residence of the president of the United States, in July 2008 The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces ... As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is elected to a four-year term via an electoral college system. Since the Twenty-second Amendment was adopted in 1951, the American presidency has been Donald Trump has officially become the 47th president of the United States. In a special ceremony called an inauguration, President Trump took over from Joe Biden as America's new leader. Donald Trump Sworn In as the 47th U.S. President Trump, 78, is the president once again after taking the oath of o

In [73]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True
)

In [74]:
#initialize the agent
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    memory=memory,
    max_iterations=3,
    early_stopping_method='generate',
    verbose=True,
)

In [77]:
response=agent.run("Do you know where is numl main islamabad is located?")



> Entering new AgentExecutor chain...
To answer this question, I should first try to find information about "NUML" and its location in Islamabad. 

Action: Duck-Duck-Search-Tool
Action Input: NUML Islamabad location
Observation: The National University of Modern Languages (also known as NUML) is a multi-campus, public research university which was established in 1970. The main campus of the NUML is situated in capital of Pakistan called Islamabad and the sub campuses are located in all four provinces of Pakistan. Choosing NUML university as Career? Things you need to know about admission's season, fee structure, departments and more information about NUML University Islamabad. National University of Modern Languages H-9 Islamabad ,Pakistan +92-51-9265100 info@numl.edu.pk About the University / University Details This University was Established in 1970, National University of Modern Languages is a non-profit public higher-education institution located in the urban setting of the metro

In [78]:
print(response)

The main campus of NUML is located in H-9, Islamabad.


In [80]:
response=agent.run("what is the capital of faranace??")



> Entering new AgentExecutor chain...
Thought: The question is asking for the capital of a country, but it seems there is a typo in the country's name. I'm going to take a guess that the country is actually "France". To confirm and find the correct answer, I'll use the Duck-Duck-Search-Tool to search for the capital of France.

Action: Duck-Duck-Search-Tool
Action Input: "capital of France"
Observation: Learn about Paris, the national capital of France, located along the Seine River in the north-central part of the country. Explore its geography, climate, culture, landmarks, and role in French history and politics. France, a country of northwestern Europe, is historically and culturally among the most important countries in the Western world. It has also played a highly significant role in international affairs for centuries. Its capital is Paris, one of the most important cultural and commercial centers in the world. Learn why Paris is the capital of France and what makes it a globa

In [81]:
print(response)

Paris


In [82]:
agent.memory.load_memory_variables({}) #model memory

{'chat_history': [HumanMessage(content='Do you know where is numl main islamabad is located?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The main campus of NUML is located in H-9, Islamabad.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is the capital of faranace??', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Paris', additional_kwargs={}, response_metadata={})]}